In [1]:
from tqdm import tqdm
import pandas as pd
from pathlib import Path
from func import ShopeeDataset, device, ShopeeNet, torch, get_train_transforms, np, f1_score_cal
from torch.utils.data import DataLoader
from sklearn.neighbors import NearestNeighbors

cuda


In [2]:
path = Path.home() / 'OneDrive - Seagroup/computer_vison/shopee_item_images/'
path_img = path / 'train_images'
train = pd.read_csv(path / 'train.csv')

df = pd.read_csv(path / 'train.csv')
tmp = df.groupby(['label_group'])['posting_id'].unique().to_dict()
df['matches'] = df['label_group'].map(tmp)
df['matches'] = df['matches'].apply(lambda x: ' '.join(x))
image_paths = str(path_img) + '/' + df['image']

In [12]:
CLASSES = 11014
BATCH_SIZE = 32
dim = (512, 512)

model_name = 'efficientnet_b3'
model = ShopeeNet(n_classes=CLASSES, model_name=model_name)
model = model.to(device)
model.eval()

image_dataset = ShopeeDataset(image_paths=image_paths.values,transforms=get_train_transforms(dim))
image_loader = DataLoader(image_dataset,
                          batch_size=BATCH_SIZE,
                          pin_memory=True,
                          num_workers=4)

embeds = []
with torch.no_grad():
    for img,label in tqdm(image_loader): 
        img = img.to(device)
        label = label.to(device)
        feat, _ = model(img,label)
        image_embeddings = feat.detach().cpu().numpy()
        embeds.append(image_embeddings)
image_embeddings = np.concatenate(embeds)

Model building for efficientnet_b3 backbone


100%|██████████████████████████████████████████████████████████████████████████████| 1071/1071 [03:55<00:00,  4.54it/s]


In [4]:
model = NearestNeighbors(n_neighbors=50)
model.fit(image_embeddings)
distances, indices = model.kneighbors(image_embeddings)

In [6]:
# Iterate through different thresholds to maximize cv, run this in interactive mode, then replace else clause with a solid threshold

thresholds = list(np.arange(0.01, 35, 1))
scores = []
for threshold in thresholds:
    predictions = []
    for k in range(image_embeddings.shape[0]):
        idx = np.where(distances[k] < threshold)[0]
        ids = indices[k, idx]
        posting_ids = ' '.join(df['posting_id'].iloc[ids].values)
        predictions.append(posting_ids)
    df['pred_matches'] = predictions
    df['f1'] = f1_score_cal(df['matches'], df['pred_matches'])
    score = df['f1'].mean()
    print(f'Our f1 score for threshold {threshold} is {score}')
    scores.append(score)

thresholds_scores = pd.DataFrame({'thresholds': thresholds, 'scores': scores})
max_score = thresholds_scores[thresholds_scores['scores'] == thresholds_scores['scores'].max()]
best_threshold = max_score['thresholds'].values[0]
best_score = max_score['scores'].values[0]
print(f'Our best score is {best_score} and has a threshold {best_threshold}')

# Use threshold
predictions = []
for k in range(image_embeddings.shape[0]):
    idx = np.where(distances[k,] < best_threshold)[0]
    ids = indices[k,idx]
    posting_ids = df['posting_id'].iloc[ids].values
    predictions.append(posting_ids)

Our f1 score for threshold 0.01 is 0.045308828548734095
Our f1 score for threshold 1.01 is 0.045308828548734095
Our f1 score for threshold 2.01 is 0.045308828548734095
Our f1 score for threshold 3.01 is 0.045308828548734095
Our f1 score for threshold 4.01 is 0.045308828548734095
Our f1 score for threshold 5.01 is 0.045308828548734095
Our f1 score for threshold 6.01 is 0.045308828548734095
Our f1 score for threshold 7.01 is 0.045308828548734095
Our f1 score for threshold 8.01 is 0.045308828548734095
Our f1 score for threshold 9.01 is 0.045308828548734095
Our f1 score for threshold 10.01 is 0.045308828548734095
Our f1 score for threshold 11.01 is 0.045308828548734095
Our f1 score for threshold 12.01 is 0.045308828548734095
Our f1 score for threshold 13.01 is 0.045308828548734095
Our f1 score for threshold 14.01 is 0.045308828548734095
Our f1 score for threshold 15.01 is 0.045308828548734095
Our f1 score for threshold 16.01 is 0.045308828548734095
Our f1 score for threshold 17.01 is 0.045

In [7]:
df['image_predictions'] = predictions

In [10]:
df.to_feather('test.ftr')

In [13]:
torch.cuda.empty_cache()